## Assess ensemble model performance on validation data for death and to output death predictions

In [19]:
import pickle
import pandas as pd
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score

#import logR death model
logR_death = pickle.load(open('ensb_logreg_death.pkl', 'rb'))

In [20]:
#load validation death probs from text and RF
validation_death_text = pd.read_csv("text_mortality_probability_validation.csv")
validation_death_text = validation_death_text[['Death', 'icu_death']]
validation_death_RF = pd.read_csv('death_probability_sd_rf_validation 1.csv')
validation_death_RF = validation_death_RF.rename(columns={"1": "RF_val_death"})
validation_death_combined = pd.concat([validation_death_text, validation_death_RF],join='outer', axis=1)
validation_death_combined

,Death,icu_death,0,RF_val_death,id
0,0.295472,0,0.197523,0.802477,20003425
1,0.219437,0,0.275247,0.724753,20008724
2,0.709828,0,0.170222,0.829778,20009330
3,0.429824,0,0.232975,0.767025,20014219
4,0.456270,0,0.176230,0.823770,20015722
...,...,...,...,...,...
4076,0.433242,0,0.136995,0.863005,29985535
4077,0.784476,1,0.195743,0.804257,29987115
4078,0.649372,0,0.234748,0.765252,29988601
4079,0.077565,0,0.246994,0.753006,29988615


In [21]:
#predict death from text and RF for validated data, and output
death_predictors = validation_death_combined[['Death','RF_val_death']]
predicted_death = logR_death.predict(death_predictors)
predicted_death_df = pd.DataFrame(predicted_death)
predicted_death_df.to_csv('180424_Validation_death.csv')

/Users/wenruiwah/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Death
- RF_val_death
Feature names seen at fit time, yet now missing:
- Death_Text
- RF_death

  warnings.warn(message, FutureWarning)


In [22]:
#scores
print('AUROC:', roc_auc_score(validation_death_combined['icu_death'], predicted_death_df))
print('Precision', precision_score(validation_death_combined['icu_death'], predicted_death_df, average='macro', zero_division=0))
print('Recall', recall_score(validation_death_combined['icu_death'], predicted_death_df, average='macro', zero_division=0))
print('F1 score:', f1_score(validation_death_combined['icu_death'], predicted_death_df, average='macro', zero_division=0))

AUROC: 0.5188807459399946
Precision 0.6691994273758397
Recall 0.5188807459399944
F1 score: 0.5110334618152637
